In [10]:
import sys
import torch

sys.path.append("..")

from models import EditorCrossAttention, EditorCrossFlashAttention2, GPT2EditorConfig

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
obj = EditorCrossFlashAttention2(GPT2EditorConfig(), layer_idx=0)

In [12]:
with torch.no_grad():
    res = obj(
        hidden_states=torch.randn(1, 2, 768),
        encoder_hidden_states=torch.randn(1, 4, 768),
    )

torch.Size([1, 4, 3072])


In [13]:
res[0].shape

torch.Size([1, 2, 768])